# Reduce Centre Cell Power

#### **Introduction**

To assess the baseline energy consumption for mobile operator networks running
5G NR SA, this document sets out a scenario for a 5G NR network comprised of a 
19 cell sites arranged in a hexagonal grid with an intersite distance of 1500m.

<!--It isn't but it should be "The scenario is based on the 5G NR SA baseline scenario defined in 3GPP TS 38.101" -->

**Scenario definition** :
[reduce_cell_centre_power.json](KISS/data/input/reduce_centre_cell_power/reduce_centre_cell_power.json).

**Source files** :
[KISS/data/output/reduce_centre_cell_power](KISS/data/output/reduce_centre_cell_power).

In [ ]:
from sys import path as sys_path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pathlib import Path
import subprocess
%matplotlib inline

# Check current working directory
path = Path.cwd()
print(f'Current directory path:{path}')

# Get the project directory
kiss_path = path.parents[1]
kiss_path_str = str(kiss_path)
print(f'Project directory path:{kiss_path}')

# Add the project directory to the system path
sys_path.append(kiss_path_str)

### **Process raw files**
Concatenate all the individual tsv files into one DataFrame.
```

In [ ]:
# Directory containing the TSV files
tsv_dir = kiss_path / 'data' / 'output' / 'reduce_centre_cell_power' / '2023_03_17' / 'Attempt_003'

# For each tsv file in the directory read the file into a DataFrame and append to a list
df_list = []
for tsv_file in tsv_dir.glob('*.tsv'):
    df_temp = pd.read_csv(tsv_file, sep='\t')
    df_list.append(df_temp)

# Concatenate the list of DataFrames into a single DataFrame
df = pd.concat(df_list)

In [ ]:
# What do the first 5 rows of the DataFrame look like?
df.head()

In [ ]:
# What does the shape look like?
df.shape

In [ ]:
# And the info?
df.info()

In [ ]:
# There is a LOT of noise here. I'll start with the basics for one DataFrame
# and then I'll try to figure out how to do it for all of them.

# Get the data for seed value 0 and sc_power(dBm) value 30.0
df_s000_p30 = df[(df['seed'] == 0) & (df['sc_power(dBm)'] == 30.0)]

# This SHOULD be a dataframe with 16 rows (+1 for the column names)
df_s000_p30.shape

In [ ]:
# Let's see whatthe whole frame looks like
display(df_s000_p30)

In [ ]:
# Reorder by ue_id
df_s000_p30_sorted = df_s000_p30.sort_values('ue_id').copy()


What we're really interested in here is the cell_throughput, cell_power, cell_ee and cell_se, which is all the same for serving_cell_id = 9 at seed = 0.

So a good summary row for serving cell 9 at seed 0 might have the columns:

seed, time, serving_cell_id, **n_ues_attached, mean_distance_to_cell, mean_ue_throughput,** sc_power, **mean_sc_rsrp, mean_sinr, mean_cqi, mean_mcs**, cell_throughput, cell_power, cell_ee, cell_se

The columns to calculate are therefore the ones above in **bold**. 

In [ ]:
# Get the number of unique ue_ids in the DataFrame
n_ues_attached = df_s000_p30_sorted['ue_id'].nunique()

print(n_ues_attached)

In [ ]:
# Now group by serving_cell_id and get the mean for distance_to_cell, ue_throughput(Mb/s), sc_rsrp, sinr, cqi and mcs
df_s000_p30_sorted_col_means = df_s000_p30_sorted.groupby("serving_cell_id")[["distance_to_cell(m)", "sc_rsrp(dBm)", "sinr(dB)", "cqi", "mcs"]].agg('mean')

In [ ]:
df_s000_p30_sorted_col_means.add_prefix('mean_')

In [ ]:
df_s000_p30_sorted_col_means.insert(0, ["seed","time","cell_id","n_ues"], [df_s000_p30[['seed','time','cell_id']], n_ues_attached])

In [ ]:
display(df_s000_p30_sorted_col_means)